**Tabla de contenido**

- [The Dataset](#The-Dataset)
- [Multilingual Transformers](#Multilingual-Transformers)
- [Una mirada más cercana a la tokenización](#Una-mirada-mas-cercana-a-la-tokenizacion)
     - [The Tokenizer Pipeline](#The-Tokenizer-Pipeline)
     - [El tokenizador SentencePiece](#El-tokenizador-SentencePiece)
- [Transformadores para Reconocimiento de Entidades Nombradas](#Transformadores-para-Reconocimiento-de-Entidades-Nombradas)
- [The Anatomy of the Transformers Model Class](#The-Anatomy-of-the-Transformers-Model-Class)
     - [Bodies and Heads](#Bodies-and-Heads)
     - [Creando un modelo personalizado para clasificación de tokens](#Creando-un-modelo-personalizado-para-clasificacion-de-tokens)
     - [Cargando un modelo personalizado](#Cargando-un-modelo-personalizado)
- [Tokenizando textos para el NER](#Tokenizando-textos-para-el-NER)
     - [Medidas de desempeño](#Medidas-de-desempeño)

Hasta ahora en este libro hemos aplicado transformadores para resolver tareas de NLP en corpora en inglés, pero `¿qué haces cuando tus documentos están escritos en griego, swahili o klingon?` Un enfoque es buscar en el Hugging Face Hub un modelo de lenguaje preentrenado adecuado y ajustarlo para la tarea en cuestión. Sin embargo, estos modelos preentrenados tienden a existir solo para lenguajes de "alto recurso" como alemán, ruso o mandarín, donde hay mucho texto web disponible para el preentrenamiento. Otro desafío común surge cuando tu corpus es multilingüe: mantener múltiples modelos monolingües en producción no será divertido ni para ti ni para tu equipo de ingeniería.

Afortunadamente, existe una clase de transformadores multilingües que vienen al rescate. Al igual que BERT, estos modelos utilizan el modelado de lenguaje enmascarado como objetivo de preentrenamiento, pero se entrenan conjuntamente en textos en más de cien idiomas. Al preentrenarse en grandes corpus a través de muchos idiomas, estos transformadores multilingües permiten la transferencia cruzada de cero disparos. Esto significa que un modelo que se ajusta finamente a un idioma puede aplicarse a otros sin ningún entrenamiento adicional. ¡Esto también hace que estos modelos sean muy adecuados para el 'cambio de código', donde un hablante alterna entre dos o más idiomas o dialectos en el contexto de una sola conversación!

En este capítulo exploraremos cómo un único modelo de transformador llamado XLM-RoBERTa (introducido en el Capítulo 3) puede ser ajustado para realizar el reconocimiento de entidades nombradas (NER) en varios idiomas. Como vimos en el Capítulo 1, el NER es una tarea común de procesamiento del lenguaje natural que identifica entidades como personas, organizaciones o ubicaciones en el texto. Estas entidades pueden ser utilizadas para diversas aplicaciones, como obtener información de documentos de empresas, mejorar la calidad de los motores de búsqueda, o simplemente construir una base de datos estructurada a partir de un corpus.

Para este capítulo, supongamos que queremos realizar NER para un cliente con sede en Suiza, donde hay cuatro idiomas nacionales (siendo el inglés a menudo un puente entre ellos). Empecemos obteniendo un corpus multilingüe adecuado para este problema.

Nota: `la Zero-shot transfer`  o el `zero-shot learning` generalmente se refiere a la tarea de entrenar un modelo en un conjunto de etiquetas y luego evaluarlo en un conjunto diferente de etiquetas. En el contexto de los transformadores, el zero-shot learning también puede referirse a situaciones en las que un modelo de lenguaje como GPT-3 es evaluado en una tarea posterior en la que ni siquiera se ha ajustado.

# The Dataset

En este capítulo, utilizaremos un subconjunto del benchmark de Evaluación de Transferencia Multilingüe de Encoders (XTREME) llamado WikiANN o PAN-X. Este conjunto de datos consiste en artículos de Wikipedia en muchos idiomas, incluidos los cuatro idiomas más hablados en Suiza: alemán (62.9%), francés (22.9%), italiano (8.4%) e inglés (5.9%).

Cada artículo está anotado con etiquetas LOC (ubicación), PER (persona) y ORG (organización) en el formato "dentro-fuera-comienzo" (IOB2). En este formato, un prefijo B indica el comienzo de una entidad, y los tokens consecutivos que pertenecen a la misma entidad reciben un prefijo I-. Una etiqueta O indica que el token no pertenece a ninguna entidad. Por ejemplo, la siguiente oración:

 `Jeff Dean is a computer scientist at Google in California` 
 
 se etiquetaría en formato IOB2 como se muestra en la siguiente Tabla.

![Tabla](images/table4.png)

Para cargar uno de los subconjuntos de PAN-X en XTREME, necesitaremos saber qué configuración de conjunto de datos pasar a la función load_dataset(). Siempre que trabajes con un conjunto de datos que tiene múltiples dominios, puedes usar la función get_dataset_config_names() para averiguar qué subconjuntos están disponibles:

In [1]:
from datasets import get_dataset_config_names
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

XTREME has 183 configurations


Vaya, ¡eso es un montón de configuraciones! Vamos a reducir la búsqueda buscando solo las configuraciones que comienzan con "PAN":

In [2]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

De acuerdo, parece que hemos identificado la sintaxis de los subconjuntos de PAN-X: cada uno tiene un sufijo de dos letras que parece ser un código de idioma ISO 639-1. Esto significa que para cargar el corpus en alemán, pasamos el código al argumento name de load_dataset() de la siguiente manera:

In [3]:
from datasets import load_dataset
load_dataset("xtreme", name="PAN-X.de") # Cargar un dataset PAN-X para un idioma

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

Para crear un corpus suizo realista, muestreamos los corpora de alemán (de), francés (fr), italiano (it) y inglés (en) de PAN-X de acuerdo con sus proporciones habladas. Esto creará un desequilibrio lingüístico que es muy común en conjuntos de datos del mundo real, donde adquirir ejemplos etiquetados en una lengua minoritaria puede ser costoso debido a la falta de expertos en la materia que hablen con fluidez ese idioma. Este conjunto de datos desequilibrado simulará una situación común al trabajar en aplicaciones multilingües, y veremos cómo podemos construir un modelo que funcione en todos los idiomas.

Para llevar un registro de cada idioma, vamos a crear un defaultdict de Python que almacene el código del idioma como clave y un corpus PAN-X del tipo DatasetDict como valor:

In [4]:
"""
Carga los datasets PAN-X para 4 idiomas (alemán, francés, italiano, inglés) y 
toma solo una fracción específica de los datos para balancearlos
"""


from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split] = (
        ds[split].shuffle(seed=0).select(range(int(frac * ds[split].num_rows)))
        )

Aquí hemos utilizado el método shuffle() para asegurarnos de que no sesgamos accidentalmente nuestras divisiones de conjuntos de datos, mientras que select() nos permite reducir la muestra de cada corpus de acuerdo con los valores en fracs. Echemos un vistazo a cuántos ejemplos tenemos por idioma en los conjuntos de entrenamiento accediendo al atributo Dataset.num_rows:

In [5]:
import pandas as pd
pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},index=["Number of training examples"])

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


Por diseño, tenemos más ejemplos en alemán que en todos los demás idiomas combinados, por lo que lo usaremos como punto de partida para realizar una transferencia cruzada multilingüe de cero disparos al francés, italiano e inglés. Inspeccionemos uno de los ejemplos en el corpus en alemán:


In [6]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


Al igual que en nuestros encuentros previos con objetos Dataset, las claves de nuestro ejemplo corresponden a los nombres de las columnas de una tabla Arrow, mientras que los valores denotan las entradas en cada columna. En particular, vemos que la columna ner_tags corresponde al mapeo de cada entidad a un ID de clase. Esto es un poco críptico para el ojo humano, así que vamos a crear una nueva columna con las etiquetas familiares LOC, PER y ORG. Para hacer esto, lo primero a notar es que nuestro objeto Dataset tiene un atributo de características que especifica los tipos de datos subyacentes asociados con cada columna:

In [7]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


La clase Sequence especifica que el campo contiene una lista de características, que en el caso de ner_tags corresponde a una lista de características ClassLabel. Seleccionemos esta característica del conjunto de entrenamiento de la siguiente manera:

In [8]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


Podemos usar el método ClassLabel.int2str() que encontramos en el Capítulo 2 para crear una nueva columna en nuestro conjunto de entrenamiento con los nombres de clase para cada etiqueta. Usaremos el método map() para devolver un diccionario con la clave correspondiente al nuevo nombre de columna y el valor como una lista de nombres de clase:

In [9]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}
panx_de = panx_ch["de"].map(create_tag_names)

Ahora que tenemos nuestras etiquetas en un formato legible para humanos, veamos cómo se alinean los tokens y las etiquetas para el primer ejemplo en el conjunto de entrenamiento.

In [10]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


La presencia de las etiquetas LOC tiene sentido ya que la frase "2,000 Einwohnern an der Danziger Bucht in der polnischen Woiwodschaft Pommern" significa "2,000 habitantes en la bahía de Gdansk en la voivodía polaca de Pomerania" en inglés, y la bahía de Gdansk es una bahía en el mar Báltico, mientras que "voivodía" corresponde a un estado en Polonia.

Como una verificación rápida de que no tenemos ningún desequilibrio inusual en las etiquetas, calculemos las frecuencias de cada entidad en cada división:


In [11]:
"""
Recorre cada partición del dataset (train, test, etc.) y cuenta cuántas veces 
aparece cada tipo de entidad (como LOC, ORG, PER) al detectar las etiquetas que empiezan por B- (inicio de entidad).
"""
from collections import Counter
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


Esto se ve bien: las distribuciones de las frecuencias de PER, LOC y ORG son aproximadamente las mismas para cada división, por lo que los conjuntos de validación y prueba deberían proporcionar una buena medida de la capacidad de generalización de nuestro etiquetador NER. A continuación, veamos algunos transformadores multilingües populares y cómo se pueden adaptar para abordar nuestra tarea de NER.

# Multilingual Transformers

Los transformadores multilingües implican arquitecturas y procedimientos de entrenamiento similares a los de sus contrapartes monolingües, excepto que el corpus utilizado para el preentrenamiento consiste en documentos en muchos idiomas. Una característica notable de este enfoque es que, a pesar de no recibir información explícita para diferenciar entre los idiomas, las representaciones lingüísticas resultantes son capaces de generalizar bien entre idiomas para una variedad de tareas posteriores. En algunos casos, esta capacidad de realizar transferencia entre idiomas puede producir resultados que son competitivos con los de los modelos monolingües, lo que elude la necesidad de entrenar un modelo por cada idioma!

Para medir el progreso de la transferencia multilingüe para el reconocimiento de entidades nombradas (NER), los conjuntos de datos CoNLL-2002 y CoNLL-2003 se utilizan a menudo como referencia para inglés, neerlandés, español y alemán. Esta referencia consiste en artículos de noticias anotados con las mismas categorías LOC, PER y ORG que PAN-X, pero contiene una etiqueta adicional MISC para entidades diversas que no pertenecen a los tres grupos anteriores. Los modelos de transformadores multilingües suelen ser evaluados de tres maneras diferentes:

- `en`: Ajustar finamente los datos de entrenamiento en inglés y luego evaluar en el conjunto de pruebas de cada idioma.
- `each`: Ajustar y evaluar en datos de prueba monolingües para medir el rendimiento por idioma.
- `all`: Ajustar finamente en todos los datos de entrenamiento para evaluar en cada conjunto de pruebas de cada idioma.

Adoptaremos una estrategia de evaluación similar para nuestra tarea de NER, pero primero necesitamos seleccionar un modelo para evaluar. Uno de los primeros transformadores multilingües fue mBERT, que utiliza la misma arquitectura y objetivo de preentrenamiento que BERT, pero añade artículos de Wikipedia de muchos idiomas al corpus de preentrenamiento. Desde entonces, mBERT ha sido superado por XLM-RoBERTa (o XLM-R para abreviar), así que ese es el modelo que consideraremos en este capítulo.

Como vimos en el Capítulo 3, XLM-R utiliza solo MLM como un objetivo de preentrenamiento para 100 idiomas, pero se distingue por el enorme tamaño de su corpus de preentrenamiento en comparación con sus precursores: volcado de Wikipedia para cada idioma y 2.5 terabytes de datos de Common Crawl de la web. Este corpus es varios órdenes de magnitud más grande que los utilizados en modelos anteriores y proporciona un impulso significativo en la señal para idiomas de bajos recursos como el birmano y el swahili, donde solo existe un pequeño número de artículos de Wikipedia.

La parte del nombre del modelo denominada RoBERTa se refiere al hecho de que el enfoque de preentrenamiento es el mismo que el de los modelos monolingües de RoBERTa. Los desarrolladores de RoBERTa mejoraron varios aspectos de BERT, en particular eliminando por completo la tarea de predicción de la siguiente oración. XLM-R también elimina las incrustaciones de lenguaje usadas en XLM y utiliza SentencePiece para tokenizar los textos sin procesar directamente. Además de su naturaleza multilingüe, una diferencia notable entre XLM-R y RoBERTa es el tamaño de sus respectivos vocabularios: 250,000 tokens frente a 55,000!

XLM-R es una excelente opción para tareas de NLU multilingües. En la próxima sección, exploraremos cómo puede tokenizar de manera eficiente en varios idiomas.


# Una mirada mas cercana a la tokenizacion

En lugar de utilizar un tokenizador WordPiece, XLM-R utiliza un tokenizador llamado SentencePiece que se entrena en el texto en bruto de los cien idiomas. Para entender cómo se compara SentencePiece con WordPiece, carguemos los tokenizadores de BERT y XLM-R de la manera habitual con Transformers:

In [12]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

Al codificar una pequeña secuencia de texto, también podemos recuperar los tokens especiales que cada modelo utilizó durante el preentrenamiento:

In [13]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

## The Tokenizer Pipeline

Hasta ahora, hemos tratado la tokenización como una sola operación que transforma cadenas en enteros que podemos pasar a través del modelo. Esto no es del todo exacto, y si miramos más de cerca, podemos ver que en realidad es un pipeline de procesamiento completo que generalmente consiste en cuatro pasos:
1. `Normalization`: Este paso corresponde al conjunto de operaciones que aplicas a una cadena de texto sin procesar para hacerla "más limpia". Las operaciones comunes incluyen eliminar espacios en blanco y quitar caracteres acentuados. La normalización de Unicode es otra operación de normalización común aplicada por muchos tokenizadores para abordar el hecho de que a menudo existen varias maneras de escribir el mismo carácter. Esto puede hacer que dos versiones de la "misma" cadena (es decir, con la misma secuencia de caracteres abstractos) parezcan diferentes; los esquemas de normalización de Unicode como NFC, NFD, NFKC y NFKD reemplazan las diversas maneras de escribir el mismo carácter con formas estándar. Otro ejemplo de normalización es convertir a minúsculas. Si se espera que el modelo acepte y utilice solo caracteres en minúscula, esta técnica se puede usar para reducir el tamaño del vocabulario que requiere. Después de la normalización, nuestra cadena de ejemplo se vería como "¡jack sparrow ama nueva york!".
2. `Pretokenization`: Este paso divide un texto en objetos más pequeños que proporcionan un límite superior a lo que serán tus tokens al final del entrenamiento. Una buena manera de pensar en esto es que el pretokenizador dividirá tu texto en "palabras", y tus tokens finales serán partes de esas palabras. Para los idiomas que permiten esto (inglés, alemán y muchos idiomas indoeuropeos), las cadenas generalmente pueden dividirse en palabras mediante espacios en blanco y puntuación. Por ejemplo, este paso puede transformar nuestro ["jack", "sparrow", "loves", "new", "york", "!"];.Estas palabras son más simples de dividir en subpalabras con algoritmos de Byte-Pair Encoding (BPE) o Unigram en el siguiente paso de la tubería. Sin embargo, dividir en "palabras" no siempre es una operación trivial y determinista, o incluso una operación que tenga sentido. Por ejemplo, en idiomas como el chino, japonés o coreano, agrupar símbolos en unidades semánticas como las palabras indoeuropeas puede ser una operación no determinista con varios grupos igualmente válidos. En este caso, podría ser mejor no pretokenizar el texto y, en su lugar, utilizar una biblioteca específica para el idioma para la pretokenización.
3. `Tokenizer model`: Una vez que los textos de entrada están normalizados y pretokens, el tokenizador aplica un modelo de división de subpalabras sobre las palabras. Esta es la parte del proceso que necesita ser entrenada en su corpus (o que ha sido entrenada si está utilizando un tokenizador preentrenado). El papel del modelo es dividir las palabras en subpalabras para reducir el tamaño del vocabulario y tratar de disminuir el número de tokens fuera de vocabulario. Existen varios algoritmos de tokenización de subpalabras, incluyendo BPE, Unigram y WordPiece. Por ejemplo, nuestro ejemplo en funcionamiento podría verse como [jack, spa, rrow, loves, new, york, !] después de aplicar el modelo tokenizador. Tenga en cuenta que en este punto ya no tenemos una lista de cadenas sino una lista de enteros (ID de entrada); para mantener el ejemplo ilustrativo, hemos conservado las palabras pero hemos quitado las comillas para indicar la transformación.
4. `Postprocessing`:Este es el último paso del proceso de tokenización, en el cual se pueden aplicar algunas transformaciones adicionales a la lista de tokens; por ejemplo, agregar tokens especiales al principio o al final de la secuencia de entrada de índices de tokens. Por ejemplo, un tokenizador estilo BERT agregaría tokens de clasificación y separación: [CLS, jack, spa, rrow, ama, nueva, york, !, SEP]. Esta secuencia (recuerda que será una secuencia de números enteros, no los tokens que ves aquí) puede ser alimentada al modelo.

Volviendo a nuestra comparación de XLM-R y BERT, ahora entendemos que SentencePiece agrega "s y \s" en lugar de [CLS] y [SEP] en el paso de posprocesamiento (como convención, continuaremos usando [CLS] y [SEP] en las ilustraciones gráficas). Volvamos al tokenizador SentencePiece para ver qué lo hace especial.

## El tokenizador SentencePiece

El tokenizador SentencePiece se basa en un tipo de segmentación de subpalabras llamada Unigram y codifica cada texto de entrada como una secuencia de caracteres Unicode. Esta última característica es especialmente útil para corpora multilingües, ya que permite que SentencePiece sea agnóstico sobre acentos, puntuación y el hecho de que muchos idiomas, como el japonés, no tienen caracteres de espacio en blanco. Otra característica especial de SentencePiece es que el espacio en blanco se asigna al símbolo Unicode U+2581, o al carácter , también llamado el carácter de bloque inferior.

Esto permite que SentencePiece des-tokenice una secuencia sin ambigüedades y sin depender de pre-tokenizadores específicos del idioma. En nuestro ejemplo de la sección anterior, por ejemplo, podemos ver que WordPiece ha perdido la información de que no hay espacio en blanco entre "York" y "!". Por el contrario, SentencePiece preserva el espacio en blanco en el texto tokenizado, por lo que podemos convertir de nuevo al texto sin procesar sin ambigüedad:

In [14]:
"".join(xlmr_tokens).replace(u"\u2581", " ")

'<s> Jack Sparrow loves New York!</s>'

Ahora que entendemos cómo funciona SentencePiece, veamos cómo podemos codificar nuestro ejemplo simple en una forma adecuada para NER. Lo primero que debe hacer es cargar el modelo previamente entrenado con un cabezal de clasificación de tokens. ¡Pero en lugar de cargar esta cabeza directamente desde los Transformadores, la construiremos nosotros mismos! Al profundizar en la API de Transformers, podemos hacer esto con solo unos pocos pasos.

# Transformadores para Reconocimiento de Entidades Nombradas

En el capítulo 2, vimos que para la clasificación de texto BERT usa el token especial [CLS] para representar una secuencia completa de texto. Esta representación se alimenta luego a través de una capa densa o completamente conectada para generar la distribución de todos los valores de etiqueta discretos, como se muestra en Figura 4-2.

![TokenizadoBert](images/tokenizadorBert.png)

Figura 4-2. *Ajuste fino de un transformador basado en codificador para la clasificación de secuencias*

BERT y otros transformadores de solo codificador adoptan un enfoque similar para NER, excepto que la representación de cada token de entrada individual se alimenta a la misma capa completamente conectada para generar la entidad del token. Por esta razón, NER a menudo se enmarca como una tarea de clasificación de tokens. El proceso se parece al diagrama de la Figura 4-3.

![Ner](images/NEr.png)

Figura 4-3. *Ajuste fino de un transformador basado en codificador para el reconocimiento de entidades nombradas*

Hasta ahora, todo bien, pero `¿cómo deberíamos manejar las subpalabras en una tarea de clasificación de tokens?` Por ejemplo, el primer nombre "Christa" en la Figura 4-3 está tokenizado en las subpalabras " Chr " y "##ist", entonces, ¿a cuál(es) se le debe asignar la etiqueta B-PER?

En el artículo de BERT, los autores asignaron esta etiqueta a la primera subpalabra ("Chr" en nuestro ejemplo) e ignoraron la siguiente subpalabra ("##ist"). Esta es la convención que adoptaremos aquí, e indicaremos las subpalabras ignoradas con IGN. Posteriormente, podemos propagar fácilmente la etiqueta predicha de la primera subpalabra a las subpalabras posteriores en el paso de posprocesamiento. También podríamos haber optado por incluir la representación de la subpalabra "# # ist " asignándole una copia de la etiqueta B-LOC, pero esto viola la Formato IOB2.

Afortunadamente, todos los aspectos de la arquitectura que hemos visto en BERT se trasladan a XLM-R ya que su arquitectura se basa en RoBERTa, que es idéntica a ¡BERT! A continuación veremos cómo Transformers admite muchas otras tareas con modificaciones menores.

# The Anatomy of the Transformers Model Class

Transformers se organiza en torno a clases dedicadas para cada arquitectura y tarea. Las clases de modelo asociadas con diferentes tareas se nombran de acuerdo con una convención<Model Name>For < Task>, o AutoModelFor < Task> cuando se usa el Clases de automodelo.

Sin embargo, este enfoque tiene sus limitaciones, y para motivar a profundizar en la API de Transformers, considere el siguiente escenario. Supongamos que tiene una gran idea para resolver un problema de PNL que ha estado en su mente durante mucho tiempo con un modelo de transformador. Entonces, programa una reunión con su jefe y, con una presentación de PowerPoint ingeniosamente elaborada, presenta que podría aumentar los ingresos de su departamento si finalmente puedes resolver el problema. Impresionado con su colorida presentación y charla sobre ganancias, su jefe acepta generosamente darle una semana para crear una prueba de concepto.

Contento con el resultado, empiezas a trabajar de inmediato. Enciendes tu GPU y abres una computadora portátil. Ejecutas desde transformers import Bert ForTaskXY (ten en cuenta que TaskXY es la tarea imaginaria que te gustaría resolver) y el color escapa de tu rostro como el temido color rojo llena tu pantalla: ImportError: no se puede importar el nombre BertForTaskXY. ¡Oh, no, no hay un modelo BERT para su caso de uso! ¿Cómo puede completar el proyecto en una semana si tiene que implementar todo el modelo usted mismo?! ¿Por dónde deberías empezar?

¡Que no cunda el pánico! Transformers está diseñado para permitirle ampliar fácilmente los modelos existentes para su caso de uso específico. Puede cargar los pesos desde modelos previamente entrenados y tiene acceso a funciones auxiliares específicas de la tarea. Esto le permite crear modelos personalizados para objetivos específicos con muy poca sobrecarga. En esta sección, veremos cómo podemos implementar nuestro propio modelo personalizado.

## Bodies and Heads

El concepto principal que hace que los Transformers sean tan versátiles es la división de la arquitectura en un cuerpo y una cabeza (como vimos en el Capítulo 1). Ya hemos visto que cuando pasamos de la tarea de preentrenamiento a la tarea específica, necesitamos reemplazar la última capa del modelo por una que sea adecuada para la tarea. Esta última capa se llama cabeza del modelo; es la parte que es específica de la tarea.

El resto del modelo se llama el cuerpo; incluye las embeddings de token y las capas de transformador que son independientes de la tarea. Este código de Transformadores también es así: la estructura del cuerpo de un modelo se refleja en su implementación en una clase como BertModel o GPT2Model que devuelve los estados ocultos de la última capa. Modelos específicos para tareas como BertForMaskedLM o BertForSequence Classification utilizan el modelo base y añaden la cabeza necesaria sobre los estados ocultos, como se muestra en la Figura 4-4.

![Bertmodel](images/Bertmodel.png)

Fig 4-4. *La clase BertModel solo contiene el cuerpo del modelo, mientras que las clases Bert For<Task> combinan el cuerpo con una cabeza dedicada para una tarea específica.*

Como veremos a continuación, esta separación de cuerpos y cabezas nos permite construir una cabeza personalizada para cualquier tarea y simplemente montarla sobre un modelo preentrenado.

## Creando un modelo personalizado para clasificacion de tokens

Vamos a realizar el ejercicio de construir una cabecera de clasificación de tokens personalizada para XLM-R. Dado que XLM-R utiliza la misma arquitectura de modelo que RoBERTa, utilizaremos RoBERTa como modelo base, pero augmentado con configuraciones específicas de XLM-R. Tenga en cuenta que este es un ejercicio educativo para mostrarle cómo construir un modelo personalizado para su propia tarea. Para la clasificación de tokens, ya existe una clase XLMRobertaForTokenClassification que puede importar de Transformers. Si lo desea, puede saltar a la siguiente sección y simplemente usar esa.

Para comenzar, necesitamos una estructura de datos que represente nuestro etiquetador XLM-R NER. Como primera suposición, necesitaremos un objeto de configuración para inicializar el modelo y una función forward() para generar los resultados. Sigamos adelante y construyamos nuestra clase XLM-R para la clasificación de tokens:

In [15]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,labels=None, **kwargs):
        # Use model body to get encoder representations
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # Return model output object
        return TokenClassifierOutput(loss=loss, 
                                     logits=logits,hidden_states=outputs.hidden_states,
                                     attentions=outputs.attentions)

La clase config_class asegura que se utilicen los ajustes estándar de XLM-R al inicializar un nuevo modelo. Si deseas cambiar los parámetros predeterminados, puedes hacerlo sobrescribiendo la configuración predeterminada. Con el método super() llamamos a la función de inicialización de la clase RobertaPreTrainedModel. Esta clase abstracta maneja la inicialización o la carga de pesos preentrenados. Luego cargamos el cuerpo de nuestro modelo, que es RobertaModel, y lo extendemos con nuestra propia cabeza de clasificación que consiste en una capa de abandono y una capa de alimentación hacia adelante estándar.

Tenga en cuenta que establecemos add_pooling_layer=False para asegurarnos de que se devuelvan todos los estados ocultos y no solo el asociado con el token [CLS]. Finalmente, inicializamos todos los pesos llamando al método init_weights() que heredamos de RobertaPreTrainedModel, que cargará los pesos preentrenados para el cuerpo del modelo y inicializará aleatoriamente los pesos de nuestra cabeza de clasificación de tokens.

Lo único que queda por hacer es definir lo que el modelo debe hacer en una pasada hacia adelante con un método forward(). Durante la pasada hacia adelante, los datos se alimentan primero a través del cuerpo del modelo. Hay una serie de variables de entrada, pero las únicas que necesitamos por ahora son input_ids y attention_mask. El estado oculto, que es parte de la salida del cuerpo del modelo, se alimenta luego a través de las capas de abandono y clasificación. Si también proporcionamos etiquetas en la pasada hacia adelante, podemos calcular directamente la pérdida. Si hay una máscara de atención, necesitamos hacer un poco más de trabajo para asegurarnos de que solo calculamos la pérdida de los tokens no enmascarados. Finalmente, envolvemos todas las salidas en un objeto TokenClassifierOutput que nos permite acceder a los elementos en una tupla nombrada familiar de capítulos anteriores.

Al implementar solo dos funciones de una clase simple, podemos construir nuestro propio modelo transformador personalizado. Y dado que heredamos de un PreTrainedModel, ¡obtendemos instantáneamente utilidades de Transformer, como from_pretrained()! Vamos a acceder a todo lo útil y veamos cómo podemos cargar pesos preentrenados en nuestro modelo personalizado.


## Cargando un modelo personalizado

Ahora estamos listos para cargar nuestro modelo de clasificación de tokens. Necesitaremos proporcionar información adicional más allá del nombre del modelo, incluyendo las etiquetas que utilizaremos para etiquetar cada entidad y el mapeo de cada etiqueta a un ID y viceversa. Toda esta información se puede derivar de nuestra variable de etiquetas, que como objeto ClassLabel tiene un atributo names que podemos usar para obtener el mapeo:

In [16]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [17]:
print(index2tag)
print(tag2index)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


Almacenaremos estas asignaciones y el atributo tags.num_classes en el objeto AutoConfig que encontramos en el Capítulo 3. Pasar argumentos de palabra clave al método from_pretrained() reemplaza los valores predeterminados:

In [18]:
from transformers import AutoConfig
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, 
                                         label2id=tag2index)

La clase AutoConfig contiene el plano de la arquitectura de un modelo. Cuando cargamos un modelo con AutoModel.from_pretrained(model_ckpt), el archivo de configuración asociado con ese modelo se descarga automáticamente. Sin embargo, si queremos modificar algo como el número de clases o los nombres de las etiquetas, entonces podemos cargar la configuración primero con los parámetros que nos gustaría personalizar.

Ahora, podemos cargar los pesos del modelo como de costumbre con la función from_pretrained() con el argumento de configuración adicional. Tenga en cuenta que no implementamos la carga de pesos preentrenados en nuestra clase de modelo personalizada; obtenemos esto de forma gratuita al heredar de RobertaPreTrainedModel:



In [19]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name, config=xlmr_config).to(device))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Como una verificación rápida de que hemos inicializado correctamente el tokenizador y el modelo, vamos a probar las predicciones en nuestra pequeña secuencia de entidades conocidas:

In [20]:
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,37456,15555,5161,7,2356,5753,38,2


Como puedes ver aquí, los tokens de inicio <s> y fin </s> se les asignan los ID 0 y 2, respectivamente.
Finalmente, necesitamos pasar las entradas al modelo y extraer las predicciones tomando el argmax para obtener la clase más probable por token:

In [21]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"Number of tokens in sequence: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

Number of tokens in sequence: 10
Shape of outputs: torch.Size([1, 10, 7])


Aquí vemos que los logits tienen la forma [tamaño_batch, num_tokens, num_tags], con cada token recibiendo un logit entre las siete posibles etiquetas de NER. Al enumerar la secuencia, podemos ver rápidamente lo que el modelo preentrenado predice:

In [22]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Tags,B-LOC,B-LOC,B-LOC,B-LOC,B-LOC,B-LOC,B-LOC,B-LOC,B-LOC,B-LOC


Como era de esperar, nuestra capa de clasificación de tokens con pesos aleatorios deja mucho que desear; ¡ajustemos un poco con datos etiquetados para mejorarla! Antes de hacerlo, envolvamos los pasos anteriores en una función auxiliar para su uso posterior:

In [23]:
def tag_text(text, tags, model, tokenizer):
    # Get tokens with special characters
    tokens = tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(input_ids)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

Antes de que podamos entrenar el modelo, también necesitamos tokenizar las entradas y preparar las etiquetas. Haremos eso a continuación.

# Tokenizando textos para el NER

Ahora que hemos establecido que el tokenizador y el modelo pueden codificar un solo ejemplo, nuestro siguiente paso es tokenizar todo el conjunto de datos para poder pasarlo al modelo XLM-R para su ajuste fino. Como vimos en el Capítulo 2, Datasets proporciona una manera rápida de tokenizar un objeto Dataset con la operación map(). Para lograr esto, recordemos que primero necesitamos definir una función con la firma mínima:

*function(examples: Dict[str, List]) -> Dict[str, List]*

donde examples es equivalente a una porción de un Conjunto de Datos, por ejemplo, panx_de['train'][:10]. Dado que el tokenizador XLM-R devuelve los ID de entrada para las entradas del modelo, solo necesitamos aumentar esta información con la máscara de atención y los ID de etiquetas que codifican la información sobre qué token está asociado con cada etiqueta NER.

Siguiendo el enfoque adoptado en cómo funciona esto con nuestro único ejemplo en alemán, recopilando primero las palabras y etiquetas como listas ordinarias:


In [24]:
words, labels = de_example["tokens"], de_example["ner_tags"]

A continuación, tokenizamos cada palabra y usamos el argumento bis_split_into_words para decirle al tokenizador que nuestra secuencia de entrada ya se ha dividido en palabras:

In [26]:
tokenized_input = xlmr_tokenizer(de_example["tokens"], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame([tokens], index=["Tokens"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


En este ejemplo podemos ver que el tokenizador ha dividido “Einwohnern” en dos subpalabras, “Einwohner” y “n”. Dado que seguimos la convención de que solo “Einwohner” debe asociarse con la etiqueta B-LOC, necesitamos una forma de enmascarar las representaciones de subpalabras después de la primera subpalabra. Afortunadamente, tokenized_input es una clase que contiene una función word_ids() que puede ayudarnos a lograr esto:


In [27]:
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=["Tokens", "Word IDs"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None


Aquí podemos ver que word_ids ha asignado cada subpalabra al índice correspondiente en la secuencia de palabras, por lo que a la primera subpalabra, "2.000", se le asigna el índice 0, mientras que a "Einwohner" y "n" se les asigna el índice 1 (ya que "Einwohnern" es la segunda palabra en palabras). También podemos ver que los tokens especiales como `<s>` y `<\s>` se asignan a None. Establezcamos –100 como la etiqueta para estas fichas especiales y las subpalabras que deseamos enmascarar durante el entrenamiento:

In [28]:
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx
labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]
pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None
Label IDs,-100,0,0,-100,0,0,5,-100,-100,6,...,5,-100,-100,-100,6,-100,-100,0,-100,-100
Labels,IGN,O,O,IGN,O,O,B-LOC,IGN,IGN,I-LOC,...,B-LOC,IGN,IGN,IGN,I-LOC,IGN,IGN,O,IGN,IGN


`¿Por qué elegimos –100 como el ID para enmascarar las representaciones de subpalabras? La razón es que en PyTorch, la clase de pérdida de entropía cruzada torch.nn.CrossEntropyLoss tiene un atributo llamado ignore_index cuyo valor es –100. Este índice se ignora durante el entrenamiento, por lo que podemos usarlo para ignorar los tokens asociados con subpalabras consecutivas.`


¡Y eso es todo! Podemos ver claramente cómo los identificadores de las etiquetas se alinean con los tokens, así que escalemos esto a todo el conjunto de datos definiendo una sola función que envuelva toda la lógica:

In [29]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True,is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Ahora tenemos todos los ingredientes que necesitamos para codificar cada división, así que escribamos una función que podamos iterar:

In [30]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True,
    remove_columns=['langs', 'ner_tags', 'tokens'])

Al aplicar esta función a un objeto DatasetDict, obtenemos un objeto Dataset codificado por división. Usemos esto para codificar nuestro corpus alemán:

In [31]:
panx_de_encoded = encode_panx_dataset(panx_ch["de"])

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Ahora que tenemos un modelo y un conjunto de datos, necesitamos definir una métrica de rendimiento.

## Medidas de desempeño

Evaluar un modelo de NER es similar a evaluar un modelo de clasificación de texto, y es común reportar resultados de precisión, recall y F1-score. La única sutileza es que todas las palabras de una entidad deben ser predichas correctamente para que una predicción se cuente como correcta. Afortunadamente, hay una excelente biblioteca llamada seqeval que está diseñada para este tipo de tareas. Por ejemplo, dado algunos tags de NER de marcador de posición y las predicciones del modelo, podemos calcular las métricas a través de la función classification_report() de seqeval:

In [ ]:
from seqeval.metrics import classification_report
